# First Try

In [54]:
import pandas as pd

imdb = pd.read_csv('data/title.basics.tsv', sep='\t')
imdb = imdb[(imdb['titleType'] == 'movie') | (imdb['titleType'] == 'tvSeries')]
imdb['primaryTitle'] = imdb['primaryTitle'].str.lower()

/tmp/ipykernel_417384/2601532259.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('data/title.basics.tsv', sep='\t')


# Get all content from Netflix in Sweden -- DEPRECATED

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# Set up the Selenium WebDriver (use appropriate WebDriver for your browser)
driver = webdriver.Chrome()

# Open the page
driver.get('https://www.justwatch.com/se/leverantör/netflix')

# Simulate scrolling
SCROLL_PAUSE_TIME = 2  # seconds

# Get the initial height of the page
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # If the height hasn't changed, we've reached the bottom of the page
    if new_height == last_height:
        break
    last_height = new_height

# Once all content is loaded, parse the page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the title grid and extract the titles
title_grid = soup.find('div', class_='title-list-grid')

if title_grid:
    title_elements = title_grid.find_all('div', {'data-title': True})
    
    titles = [element['data-title'] for element in title_elements]
    
    for title in titles:
        print(title)
else:
    print("No titles found.")

# Close the driver
driver.quit()


# Scrap IMDB

In [53]:
netflix = pd.read_csv('data/netflix_titles.csv')
netflix['type'] = netflix['type'].apply(lambda x: 'movie' if x == 'Movie' else 'tvSeries')
netflix['release_year'] = netflix['release_year'].astype(str)
netflix['title'] = netflix['title'].str.lower()

In [55]:
df = pd.merge(imdb, netflix, left_on=['primaryTitle', 'startYear'], right_on=['title', 'release_year'], how='inner')

In [56]:
print(netflix.shape)
print(df.shape)

(8807, 12)
(5551, 21)


In [63]:
df[df['title'] == 'office']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,show_id,...,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [62]:
netflix[netflix['title'].str.contains('office')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
316,s317,tvSeries,office girls,Hsu Fu-chun,"Roy Chiu, Alice Ko, Tia Lee, Patrick Lee, Kuo ...",Taiwan,"August 3, 2021",2011,TV-14,1 Season,"International TV Shows, Romantic TV Shows, TV ...",A department store mogul has his son work inco...
4535,s4536,tvSeries,yg future strategy office,NaN,"Seungri, Yoo Byung Jae, Lee Jai-jin, Jinu, Son...",South Korea,"October 5, 2018",2018,TV-MA,1 Season,"International TV Shows, Korean TV Shows, TV Co...","K-pop star Seungri, BIGBANG's youngest member,..."
7335,s7336,movie,"look out, officer",Sze Yu Lau,"Stephen Chow, Bill Tung, Stanley Sui-Fan Fung,...",Hong Kong,"August 16, 2018",1990,TV-14,88 min,"Action & Adventure, Comedies, International Mo...",An officer killed on the job returns to Earth ...
7642,s7643,movie,officer downe,Shawn Crahan,"Kim Coates, Tyler Ross, Lauren Vélez, Reno Wil...",United States,"March 30, 2017",2016,R,91 min,Action & Adventure,Fallen Los Angeles police officer Terrence Dow...
8442,s8443,tvSeries,the office (u.s.),NaN,"Steve Carell, John Krasinski, Jenna Fischer, R...",United States,"January 1, 2016",2012,TV-14,9 Seasons,TV Comedies,This hit comedy chronicles the foibles of disg...
8446,s8447,movie,the parole officer,John Duigan,"Steve Coogan, Emma Gilmour, Susan Jane Tanner,...",United Kingdom,"August 1, 2018",2001,R,94 min,"Action & Adventure, Comedies","After witnessing a murder, Simon Garden is fra..."


In [66]:
imdb[imdb['tconst'] == 'tt0386676']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
370267,tt0386676,tvSeries,the office,The Office,0,2005,2013,22,Comedy
